In [ ]:
import tensorflow as tf
print(len(tf.config.list_physical_devices('GPU')))

In [ ]:
import keras
import numpy as np
from keras import backend as K
from keras import layers as L
from keras import losses
from keras.models import Model
from sklearn.model_selection import train_test_split

In [93]:
from skimage import io
import os

list_images = os.listdir("F:\College\Seventh\GANS\Sec\CNN\data1")
data = []
for i in list_images:
    data.append(io.imread("F:\College\Seventh\GANS\Sec\CNN\data1\\" + i))
data = np.array(data)
data[0].shape

(224, 224, 3)

In [94]:
latent_dim = 10
original_dim = 224 * 224 * 3

In [95]:
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon


In [96]:
input_img = L.Input(shape=(224, 224, 3))

############
# Encoding #
############

x = L.Conv2D(128, (3, 3), activation='relu', padding='same')(input_img)
x = L.MaxPooling2D((2, 2), padding='same')(x)
x = L.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = L.MaxPooling2D((2, 2), padding='same')(x)
x = L.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = L.MaxPooling2D((2, 2), padding='same')(x)
x = L.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = L.MaxPooling2D((2, 2), padding='same')(x)
x = L.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = L.Flatten()(x)
h = L.Dense(32, activation='relu')(x)
z_mean = L.Dense(latent_dim)(h)
z_log_sigma = L.Dense(latent_dim)(h)
z = L.Lambda(sampling)([z_mean, z_log_sigma])
encoder = keras.Model(input_img, [z_mean, z_log_sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_70 (Conv2D)             (None, 224, 224, 12  3584        ['input_15[0][0]']               
                                8)                                                                
                                                                                                  
 max_pooling2d_56 (MaxPooling2D  (None, 112, 112, 12  0          ['conv2d_70[0][0]']              
 )                              8)                                                          

In [97]:
############
# Decoding #
############
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x=L.Dense(14*14*16,activation="relu")(latent_inputs)
x = L.Reshape((14,14,16))(x)
x = L.Conv2DTranspose(128, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
x = L.Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
x = L.Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
x = L.Conv2DTranspose(3, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
decoder = keras.Model(latent_inputs, x, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 10)]              0         
                                                                 
 dense_55 (Dense)            (None, 3136)              34496     
                                                                 
 reshape_9 (Reshape)         (None, 14, 14, 16)        0         
                                                                 
 conv2d_transpose_33 (Conv2D  (None, 28, 28, 128)      18560     
 Transpose)                                                      
                                                                 
 conv2d_transpose_34 (Conv2D  (None, 56, 56, 64)       73792     
 Transpose)                                                      
                                                                 
 conv2d_transpose_35 (Conv2D  (None, 112, 112, 32)     1846

In [98]:
outputs = decoder(encoder(input_img )[2])
VAE = Model(input_img, outputs)
VAE.compile(optimizer='adam', loss='mean_squared_error')
VAE.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 encoder (Functional)        [(None, 10),              203828    
                              (None, 10),                        
                              (None, 10)]                        
                                                                 
 decoder (Functional)        (None, 224, 224, 3)       146179    
                                                                 
Total params: 350,007
Trainable params: 350,007
Non-trainable params: 0
_________________________________________________________________


In [99]:
reconstruction_loss = keras.losses.binary_crossentropy(input_img , outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
VAE.add_loss(vae_loss)

In [100]:
x_train=x_test = data[0:51]


x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

(51, 224, 224, 3)
(51, 224, 224, 3)


In [ ]:
VAE.fit(x_train,x_train,epochs=3,batch_size=10)

Epoch 1/3
